In [1]:
import csv
from pprint import pprint

file_path = "./data/24.csv"

data_dict = {}

with open(file_path, "r") as f:
    csv_reader = csv.reader(f)
    keys = next(csv_reader)
    for row in csv_reader:
        data_dict[row[0]] = dict(zip(keys, row))

puzzle = data_dict['1']['Puzzles']
pprint(puzzle)

'1 1 4 6'


In [2]:
import guidance
from guidance import models, gen, select, one_or_more

lm = models.Transformers("microsoft/phi-2", temperature=0.5)

/mnt/d/Projects/forge-notebooks/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/d/Projects/forge-notebooks/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/mnt/d/Projects/forge-notebooks/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sa

In [3]:
@guidance
def add(lm, input1, input2):
    lm += f' = {int(input1) + int(input2)}'
    return lm

@guidance
def subtract(lm, input1, input2):
    lm += f' = {int(input1) - int(input2)}'
    return lm

@guidance
def multiply(lm, input1, input2):
    lm += f' = {float(input1) * float(input2)}'
    return lm

@guidance
def divide(lm, input1, input2):
    lm += f' = {float(input1) / float(input2)}'
    return lm

In [4]:
query = f"Use numbers and basic arithmetic operations (+ - * /) to obtain 24.\nPuzzle: {puzzle}\n"

puzzle_state = {
    'puzzle': [int(char) for char in puzzle.split()],
    'max_attempts': 10,
    'history': [],
    'solved': False,
    'solution': None
}

i = 0

while i < puzzle_state['max_attempts']:
    
    left = puzzle_state['puzzle'].copy()
    history = '\n'.join([' '.join(h) for h in puzzle_state['history']])
    attempt = []
    print(left)
    
    while len(left) > 1:
        iteration = lm + query + history + "\nStep: " 
        
        iteration += select(left, name='num1')
        left.pop(left.index(iteration['num1']))

        iteration += select(['+' , '-', '*', '/'], name='operation')
        
        iteration += select(left, name='num2')
        left.pop(left.index(iteration['num2']))
        
        iteration += "=" + gen(tools=[add, subtract, multiply, divide], stop=['\n', ','], regex='\\s*[+-]?\\d+(\\.\\d{1,5})?\\s*$', name='total')
        left.append(iteration['total'])

        step = f"Step: {iteration['num1']} {iteration['operation']} {iteration['num2']} = {iteration['total']} "
        state = f"Left: {left}"
        
        action = step + state + "\n"
        attempt.append(action.strip())
    
    answer = "Answer: " + left[0] + "\n"
    attempt.append(answer.strip())
    
    if float(left[0]) == 24:
        puzzle_state['solved'] = True
        puzzle_state['solution'] = attempt
        break
        
    else:
        attempt.append(("This is incorrect. Try again.\n\n").strip())
        puzzle_state['history'].append(attempt)
        i += 1

In [6]:
puzzle_state

{'puzzle': [1, 1, 4, 6],
 'max_attempts': 10,
 'history': [["Step: 1 + 1 = 2 Left: ['4', '6', '2']",
   "Step: 4 + 6 = 10 Left: ['2', '10']",
   "Step: 10 - 2 = 8 Left: ['8']",
   'Answer: 8',
   'This is incorrect. Try again.'],
  ["Step: 1 / 4 = 0.25 Left: ['1', '6', '0.25']",
   "Step: 1 / 6 = 0.1666 Left: ['0.25', '0.1666']",
   "Step: 0.1666 * 0.25 = 0.0416 Left: ['0.0416']",
   'Answer: 0.0416',
   'This is incorrect. Try again.'],
  ["Step: 1 / 4 = 0.25 Left: ['1', '6', '0.25']",
   "Step: 1 / 6 = 0.1666 Left: ['0.25', '0.1666']",
   "Step: 0.25 / 0.1666 = 1.5125 Left: ['1.5125']",
   'Answer: 1.5125',
   'This is incorrect. Try again.'],
  ["Step: 1 / 4 = 0.25 Left: ['1', '6', '0.25']",
   "Step: 1 / 6 = 0.1666 Left: ['0.25', '0.1666']",
   "Step: 0.25 / 0.1666 = 1.5125 Left: ['1.5125']",
   'Answer: 1.5125',
   'This is incorrect. Try again.'],
  ["Step: 1 / 4 = 0.25 Left: ['1', '6', '0.25']",
   "Step: 1 / 6 = 0.1666 Left: ['0.25', '0.1666']",
   "Step: 0.25 / 0.1666 = 1.5125

In [ ]:
query = f"Use numbers and basic arithmetic operations (+ - * /) to obtain 24. {puzzle}"

puzzle_list = [int(char) for char in puzzle.split()]
history = None
total = None

objective = context['objective']

rules = ""

strategies = ""

game_of_24 = lm + f"""\
Query: {query} \n
Objective: {objective} \n
Rules: {rules} \n
Strategies: {strategies} \n
History: {history} \n
Total: {total}
"""

game_of_24

## References from guidance-ai docs.

In [ ]:
@guidance
def react_prompt_example(lm, question, max_rounds=10):
    lm += f'Question: {question}\n'
    i = 1
    while True:
        lm += f'Thought {i}: ' + gen(suffix='\n')
        lm += f'Act {i}: ' + select(['Search', 'Finish'], name='act') 
        lm += '[' + gen(name='arg', suffix=']') + '\n'
        if lm['act'] == 'Finish' or i == max_rounds:
            break
        else:
            lm += f'Observation {i}: ' + search(lm['arg']) + '\n'
        i += 1
    return lm

In [ ]:
import guidance
from guidance import one_or_more, select, zero_or_more

# stateless=True indicates this function does not depend on LLM generations
@guidance(stateless=True)
def number(lm):
    n = one_or_more(select(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']))
    # Allow for negative or positive numbers
    return lm + select(['-' + n, n])

@guidance(stateless=True)
def operator(lm):
    return lm + select(['+' , '*', '**', '/', '-'])

@guidance(stateless=True)
def expression(lm):
    # Either
    # 1. A number (terminal)
    # 2. two expressions with an operator and optional whitespace
    # 3. An expression with parentheses around it
    return lm + select([
        number(),
        expression() + zero_or_more(' ') +  operator() + zero_or_more(' ') +  expression(),
        '(' + expression() + ')'
    ])